
![](http://www.travelguru.com/holiday-packages/images/goa.jpg)

# Table of contents

i) Introduction

ii) Dataset preparation.

iii) Preprocessing of data.

iv) Applying Cosine similarity.

v) Recommending Places.

#  i Introduction

Ever wondered how Amazon is suggesting your favourites shoes or how youtube is suggesting a video which you like to watch. Recommender systems are the reason for this. The recommender systems are divided into three as of now

* Popularity based recommendation engine
* Content based recommendation engine
* Collaborative filtering based recommendation engine
.

### 1)Popularity based recommendation engine:

Thisis the simplest kind of recommendation engine. The trending list you see in YouTube or Netflix is based on this algorithm. It keeps a track of view counts for each movie/video and then lists movies based on views in descending order(highest view count to lowest view count).

### 2)Content based recommendation engine:

Lets see this with the help of an example.Consider someone likes a movie. The recommender system takes it. Then it analyzes the contents (storyline, genre, cast, director etc.) of the movie to find out other movies which have similar content. Then it ranks similar movies according to their similarity scores and recommends the most relevant movies to the user.

### 3)Collaborative filtering based recommendation engine:

This algorithm at first tries to find similarity between users based on their activities and preferences (for example, both the users watch same type of movies or movies directed by the same director). Now, between these users(say, A and B) if user A has seen a movie that user B has not seen yet, then that movie gets recommended to user B and vice-versa. In other words, the recommendations get filtered based on the collaboration between similar user’s preferences (thus, the name “Collaborative Filtering”).ex: Amazon e-commerce platform, where you get to see the “Customers who viewed this item also viewed”.



This notebook is a content based recommender engine. It takes in different details of travel packages, analyses it and predicts new travel packages based on the itinery details.

# ii Dataset preparation

In [ ]:
import warnings
warnings.filterwarnings("ignore")
#importing the packages needed
import pandas as pd
import numpy as np
import re
import nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#reading the dataset scraped from different websites

df3=pd.read_csv("/content/drive/MyDrive/dataset/agra.csv")
df26=pd.read_csv("/content/drive/MyDrive/dataset/Amritsar.csv")
df1=pd.read_csv("/content/drive/MyDrive/dataset/bengaluru.csv")
df4=pd.read_csv("/content/drive/MyDrive/dataset/Chennai.csv")
df6=pd.read_csv("/content/drive/MyDrive/dataset/Delhi.csv")
df23 =pd.read_csv("/content/drive/MyDrive/dataset/Dalhousie.csv")
df5=pd.read_csv("/content/drive/MyDrive/dataset/Goa.csv")
df8=pd.read_csv("/content/drive/MyDrive/dataset/Jaipur.csv")
df9=pd.read_csv("/content/drive/MyDrive/dataset/Jaisalmer.csv")
df10=pd.read_csv("/content/drive/MyDrive/dataset/Jodhpur.csv")
df7=pd.read_csv("/content/drive/MyDrive/dataset/HimachalPradesh.csv")
df24 =pd.read_csv("/content/drive/MyDrive/dataset/Kanyakumari.csv")
df11=pd.read_csv("/content/drive/MyDrive/dataset/Kochi.csv")
df2=pd.read_csv("/content/drive/MyDrive/dataset/Kerala.csv")
df12=pd.read_csv("/content/drive/MyDrive/dataset/Madurai.csv")
df13=pd.read_csv("/content/drive/MyDrive/dataset/Manali.csv")
df14=pd.read_csv("/content/drive/MyDrive/dataset/Munnar.csv")
df15=pd.read_csv("/content/drive/MyDrive/dataset/Nepal.csv")
df16=pd.read_csv("/content/drive/MyDrive/dataset/Ooty.csv")
df17=pd.read_csv("/content/drive/MyDrive/dataset/PortBlair.csv")
df18=pd.read_csv("/content/drive/MyDrive/dataset/Puducherry.csv")
df25=pd.read_csv("/content/drive/MyDrive/dataset/Rajasthan.csv")
df19=pd.read_csv("/content/drive/MyDrive/dataset/Shimla.csv")
df20=pd.read_csv("/content/drive/MyDrive/dataset/Sikkim.csv")
df21=pd.read_csv("/content/drive/MyDrive/dataset/Tirupati.csv")
df22=pd.read_csv("/content/drive/MyDrive/dataset/udaipur.csv")



In [ ]:
#concatenating all the dataframes to a single one
frames = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20,df21,df22,df23,df24,df25,df26]
df = pd.concat(frames)
#dropping all the row where the about section is empty
#This is imp as we are analysing the cosine similarity of this section
data=df.dropna(axis = 0, how ='any')

In [ ]:
df.shape

(253, 6)

In [ ]:
print(df.iloc[:,0])

0    Bengaluru
1    Bengaluru
2    Bengaluru
3    Bengaluru
4    Bengaluru
       ...    
5     Amritsar
6     Amritsar
7     Amritsar
8     Amritsar
9     Amritsar
Name: Place, Length: 253, dtype: object


# iii Section 2 -- Preprocessing of data

In [ ]:
#Joining the columns to make a unique text column which defines a package --- The cosine similarity of this column is used to predict recommendations
features = ['Place','Pack_Name','About_trip']
def combine_features(row):
 return row['Place']+" "+row['Pack_Name']+" "+row['About_trip']
for feature in features:
    data[feature] = data[feature].fillna('') #filling all NaNs with blank string
data["combined_features"] = data.apply(combine_features,axis=1) #applying combined_features() method over each rows of dataframe and storing the combined string in “combined_features” column
#data.head(50) ---seeing the data

In [ ]:
#downloading stopwords to remove the common words
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Function to do basic text cleaning
def clean(text):

    # Urls
    text = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", text)

    # Words with punctuations and special characters
    punctuations =['@','#','!','?','+','&','*','[',']','-','%','.',':','/','(',')',';','$','=','>','<','|','{','}','^']
    for p in punctuations:
        text = text.replace(p, f' {p} ')

    return text

In [ ]:
data.iloc[6] = data.iloc[6].apply(lambda s : clean(s)) #cleaning the text using above function

In [ ]:
#removing stopwords
from nltk.corpus import stopwords
data["combined_features"] = data["combined_features"].str.lower().str.split()
stop = stopwords.words('english')
data['combined_features']=data['combined_features'].apply(lambda x: [item for item in x if item not in stop])
data["combined_features"]= data["combined_features"].str.join(" ") #rejoining the words to text

In [ ]:
# defining a new column with no's and setting it as index
ind=[]
for i in range(253):
     ind.append(i)

data.insert (0,"index",ind)
data.set_index(['index'])

,Place,Pack_Name,Price,Time,About_trip,Package Inclusions,combined_features
index,,,,,,,
0,Bengaluru,Memorable Karnataka & Kerala,"rs 21,399",7 Nights / 8 Days,Detailed Day Wise Itinerary\nDay 1 - Arrive in...,Memorable Karnataka & Kerala,bengaluru memorable karnataka & kerala detaile...
1,Bengaluru,Pleasant Coorg & Munnar Standard,"rs 15,950",8 Nights / 9 Days,Detailed Day Wise Itinerary\nDay 1 - Arrival i...,Pleasant Coorg & Munnar Standard,bengaluru pleasant coorg & munnar standard det...
2,Bengaluru,5 Nights South Tour by Own Car,"rs 8,999",5 Nights / 6 Days,Detailed Day Wise Itinerary\nDay 1 - Bengaluru...,5 Nights South Tour by Own Car,bengaluru 5 nights south tour car detailed day...
3,Bengaluru,Together in Coorg and Ooty - With Flights,"rs 20,500",5 Nights / 6 Days,Detailed Day Wise Itinerary\nDay 1 - Arrive in...,Together in Coorg and Ooty - With Flights,bengaluru together coorg ooty - flights detail...
4,Bengaluru,Nature Rendezvous - Deluxe,"rs 35,354",4 Nights / 5 Days,Detailed Day Wise Itinerary\nDay 1 - Bangalore...,Nature Rendezvous - Deluxe,bengaluru nature rendezvous - deluxe detailed ...
...,...,...,...,...,...,...,...
248,Amritsar,Picturesque Dalhousie - Land Only (Ex Delhi),"rs 15,199",4 Nights / 5 Days,Detailed Day Wise Itinerary\nDay 1 - Drive fro...,Picturesque Dalhousie - Land Only (Ex Delhi),amritsar picturesque dalhousie - land (ex delh...
249,Amritsar,Charming Dalhousie - Ex Delhi,"rs 16,899",3 Nights / 4 Days,Detailed Day Wise Itinerary\nDay 1 - Drive fro...,Charming Dalhousie - Ex Delhi,amritsar charming dalhousie - ex delhi detaile...
250,Amritsar,Fabulous Himachal with Amritsar Ex Delhi Land ...,"rs 28,650",8 Nights / 9 Days,Detailed Day Wise Itinerary\nDay 1 - Drive fro...,Fabulous Himachal with Amritsar Ex Delhi Land ...,amritsar fabulous himachal amritsar ex delhi l...


In [ ]:
# data
# data.dropna()
# data1 = data.drop(0)
# data1
print(data.iloc[:,0])

0      0
1      1
2      2
3      3
4      4
    ... 
5    248
6    249
7    250
8    251
9    252
Name: index, Length: 253, dtype: int64


In [ ]:
from google.colab import files
data.to_csv("file2.csv")
files.download("file2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
data.describe()

,index
count,253.000000
mean,126.000000
std,73.179004
min,0.000000
25%,63.000000
50%,126.000000
75%,189.000000
max,252.000000


In [ ]:
data.head()

,index,Place,Pack_Name,Price,Time,About_trip,Package Inclusions,combined_features
0,0,Bengaluru,Memorable Karnataka & Kerala,"rs 21,399",7 Nights / 8 Days,Detailed Day Wise Itinerary\nDay 1 - Arrive in...,Memorable Karnataka & Kerala,bengaluru memorable karnataka & kerala detaile...
1,1,Bengaluru,Pleasant Coorg & Munnar Standard,"rs 15,950",8 Nights / 9 Days,Detailed Day Wise Itinerary\nDay 1 - Arrival i...,Pleasant Coorg & Munnar Standard,bengaluru pleasant coorg & munnar standard det...
2,2,Bengaluru,5 Nights South Tour by Own Car,"rs 8,999",5 Nights / 6 Days,Detailed Day Wise Itinerary\nDay 1 - Bengaluru...,5 Nights South Tour by Own Car,bengaluru 5 nights south tour car detailed day...
3,3,Bengaluru,Together in Coorg and Ooty - With Flights,"rs 20,500",5 Nights / 6 Days,Detailed Day Wise Itinerary\nDay 1 - Arrive in...,Together in Coorg and Ooty - With Flights,bengaluru together coorg ooty - flights detail...
4,4,Bengaluru,Nature Rendezvous - Deluxe,"rs 35,354",4 Nights / 5 Days,Detailed Day Wise Itinerary\nDay 1 - Bangalore...,Nature Rendezvous - Deluxe,bengaluru nature rendezvous - deluxe detailed ...


In [ ]:
data.tail()

,index,Place,Pack_Name,Price,Time,About_trip,Package Inclusions,combined_features
5,248,Amritsar,Picturesque Dalhousie - Land Only (Ex Delhi),"rs 15,199",4 Nights / 5 Days,Detailed Day Wise Itinerary\nDay 1 - Drive fro...,Picturesque Dalhousie - Land Only (Ex Delhi),amritsar picturesque dalhousie - land (ex delh...
6,249,Amritsar,Charming Dalhousie - Ex Delhi,"rs 16,899",3 Nights / 4 Days,Detailed Day Wise Itinerary\nDay 1 - Drive fro...,Charming Dalhousie - Ex Delhi,amritsar charming dalhousie - ex delhi detaile...
7,250,Amritsar,Fabulous Himachal with Amritsar Ex Delhi Land ...,"rs 28,650",8 Nights / 9 Days,Detailed Day Wise Itinerary\nDay 1 - Drive fro...,Fabulous Himachal with Amritsar Ex Delhi Land ...,amritsar fabulous himachal amritsar ex delhi l...
8,251,Amritsar,Charming Dalhousie - Ex Delhi (With Flight),"rs 28,750",3 Nights / 4 Days,Detailed Day Wise Itinerary\nDay 1 - Drive fro...,Charming Dalhousie - Ex Delhi (With Flight),amritsar charming dalhousie - ex delhi (with f...
9,252,Amritsar,Dharamshala & Dalhousie Getaway - Ex Delhi (Wi...,"rs 32,749",5 Nights / 6 Days,Detailed Day Wise Itinerary\nDay 1 - Drive fro...,Dharamshala & Dalhousie Getaway - Ex Delhi (Wi...,amritsar dharamshala & dalhousie getaway - ex ...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253 entries, 0 to 9
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   index               253 non-null    int64 
 1   Place               253 non-null    object
 2   Pack_Name           253 non-null    object
 3   Price               253 non-null    object
 4   Time                253 non-null    object
 5   About_trip          253 non-null    object
 6   Package Inclusions  253 non-null    object
 7   combined_features   253 non-null    object
dtypes: int64(1), object(7)
memory usage: 17.8+ KB


This marks the end of our preprocessing of data.

## Section 3: Applying Cosine Similarity

As I said earlier we are going to make a content based recommender system. So we are using the combined_features column. We are calculating the cosine similarity on this column.

Cosine similarity is a metric used to determine how similar the documents are irrespective of their size.
Mathematically, it measures the cosine of the angle between two vectors projected in a multi-dimensional space. In this context, the two vectors I am talking about are arrays containing the word counts of two documents

When plotted on a multi-dimensional space, where each dimension corresponds to a word in the document, the cosine similarity captures the orientation (the angle) of the documents and not the magnitude. If you want the magnitude, compute the Euclidean distance instead.

![](http:///www.researchgate.net/publication/320914786/figure/fig2/AS:558221849841664@1510101868614/The-difference-between-Euclidean-distance-and-cosine-similarity.png)

In [ ]:
#import modules
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#data.rePlace('', np.nan, inPlace=True)
cv = CountVectorizer() #creating new CountVectorizer() object
count_matrix = cv.fit_transform(data['combined_features'].values.astype('U'))  #fitting cv object to combine features column
cosine_sim = cosine_similarity(count_matrix) #calculating cosine similarity

## Section 4: Recommending Places

In [ ]:
#creating new column to merge Place and Pack_Name which serves as user input identifier
data.head()
features2 = ['Place','Pack_Name']
def combine_features2(row):
 return row['Place']+" "+row['Pack_Name']
for feature in features2:
    data[feature] = data[feature].fillna('') #filling all NaNs with blank string
data["Place_names"] = data.apply(combine_features,axis=1)
data["Place_names"] = data["Place_names"].str.lower()

In [ ]:
data.head()

,index,Place,Pack_Name,Price,Time,About_trip,Package Inclusions,combined_features,Place_names
0,0,Bengaluru,Memorable Karnataka & Kerala,"rs 21,399",7 Nights / 8 Days,Detailed Day Wise Itinerary\nDay 1 - Arrive in...,Memorable Karnataka & Kerala,bengaluru memorable karnataka & kerala detaile...,bengaluru memorable karnataka & kerala detaile...
1,1,Bengaluru,Pleasant Coorg & Munnar Standard,"rs 15,950",8 Nights / 9 Days,Detailed Day Wise Itinerary\nDay 1 - Arrival i...,Pleasant Coorg & Munnar Standard,bengaluru pleasant coorg & munnar standard det...,bengaluru pleasant coorg & munnar standard det...
2,2,Bengaluru,5 Nights South Tour by Own Car,"rs 8,999",5 Nights / 6 Days,Detailed Day Wise Itinerary\nDay 1 - Bengaluru...,5 Nights South Tour by Own Car,bengaluru 5 nights south tour car detailed day...,bengaluru 5 nights south tour by own car detai...
3,3,Bengaluru,Together in Coorg and Ooty - With Flights,"rs 20,500",5 Nights / 6 Days,Detailed Day Wise Itinerary\nDay 1 - Arrive in...,Together in Coorg and Ooty - With Flights,bengaluru together coorg ooty - flights detail...,bengaluru together in coorg and ooty - with fl...
4,4,Bengaluru,Nature Rendezvous - Deluxe,"rs 35,354",4 Nights / 5 Days,Detailed Day Wise Itinerary\nDay 1 - Bangalore...,Nature Rendezvous - Deluxe,bengaluru nature rendezvous - deluxe detailed ...,bengaluru nature rendezvous - deluxe detailed ...


In [ ]:
# print(data.iloc[:,-1])
st = ""
for i in range(253):
  st += data.iloc[i]['Place_names']
# print(st)
tt = re.search("ooty",st)
find = re.findall("ooty",st)
print(len(find))
print(tt)
# ['Place_names'])
print(data.index)

83
<re.Match object; span=(727, 731), match='ooty'>
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
            ...
            0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
           dtype='int64', length=253)


In [ ]:
#function to search user input on the newly created column and return the index
import re
def get_index_from_title(Place):
    # count = 0
    for ind in range(253):
        mond=data.iloc[ind]['Place_names']
        # p
        # mond = mond.lower()
        # count += 1
        # print("index :  {}   {}".format(ind,mond))
        if re.search(Place,mond):
            return(ind)
    # print(count)


In [ ]:
#finding similar Places using cosine similarity
Place_selec = input('Enter place: ')
# print(Place_selec)
Place_index = get_index_from_title(Place_selec)
# print(Place_index)
# print(cosine_similarity)
similar_Places = list(enumerate(cosine_sim[Place_index])) #accessing the row corresponding to given Place name to find all the similarity scores for that Place name and then enumerating over it
# similar_Places = list(enumerate(cosine_sim[Place_index]))
similar_Places = list(set(similar_Places))
print(similar_Places)
# print(len(similar_Places))
#sorting those packageso in descending order
# cost = sorted(similar_Places,key = lambda x:x[1)
sorted_similar_Places = sorted(similar_Places,key=lambda x:x[1],reverse=True)[1:]
# sorted_similar_Places = list(set(sorted_similar_Places))
print(sorted_similar_Places)


#printing the top 5 recommendations
i=0
print()
print()
print("Top 5 similar travel packages like "+Place_selec+" are:\n")
for element in sorted_similar_Places:
    print("Package name: {}".format(data.loc[data['index'] == element[0], 'Pack_Name'].values[0]))
    print("Place: {}".format(data.loc[data['index'] == element[0], 'Place'].values[0]))
    dur=data.loc[data['index'] == element[0], 'Time'].values[0]
    print("Duration:{}".format(" ".join(dur.split()))) #removing extra Places in emi column.
    print("Amount: Rs {} .".format(data.loc[data['index'] == element[0], 'Price'].values[0]))
    plan = data.loc[data['index'] == element[0], 'About_trip'].values[0]
    print("Detailed itenary {} . ".format(" ".join(plan.split())))
    # print('Emi amount: Rs {}'.format(data.loc[data['index'] == element[0], 'emi'].values[0]))
    print()
    print()
    i=i+1
    if i>5:
        break

Enter place: ooty
[(52, 0.42132896272590786), (24, 0.4191824503674153), (8, 0.43312727300549675), (62, 0.4246773250216147), (50, 0.504276383391484), (128, 0.28348255888552515), (6, 0.47081366596560026), (28, 0.36054417107205283), (180, 0.5410966019294753), (209, 0.3108622182000963), (39, 0.3072809356941946), (234, 0.44849416675108955), (122, 0.4117399428817459), (78, 0.565478782304887), (223, 0.5305061317948665), (149, 0.4800190966546775), (210, 0.565478782304887), (93, 0.565478782304887), (189, 0.3675874419924318), (220, 0.48679476243288416), (116, 0.41071443707296507), (22, 0.21608184806056946), (127, 0.36464302238750285), (250, 0.5380497896107538), (14, 0.6285816912134299), (193, 0.28488221017533083), (17, 0.5411636285371207), (106, 0.27223939596701435), (251, 0.3617768547571534), (233, 0.5119662239845131), (36, 0.44558499540244056), (212, 0.2788665559950094), (226, 0.4107144370729652), (232, 0.30165638331520417), (125, 0.39367834475995844), (241, 0.26874918548692905), (184, 0.24894